# CP 2 Aprendizaje de Máquinas
---
## Clasificación supervisada e introducción a la herramienta sklearn

El objetivo de la clase práctica es resolver un problema utilizando los algoritmos dados en conferencia. Se tienen críticas de cine positivas y negativas. 1000 de cada una. El objetivo es, dado el texto de la crítica, determinar si la crítica es positiva o negativa.

### Ejercicio 1: Preprocesamiento de los datos

In [8]:
import os
from pathlib import Path
from typing import List

En la carpeta `txt_sentoken` se encuentran las críticas. Dentro hay dos carpetas (`pos`, `neg`), donde están las críticas positivas y negativas.

In [9]:
path_p = Path("txt_sentoken/pos")
path_n = Path("txt_sentoken/neg")

Cada archivo de cada carpeta es una crítica diferente, por lo que tenemos que analizar los archivos de cada una de estas carptetas para procesar las críticas.

In [10]:
ds_p = list(path_p.iterdir())     # directorio donde están las críticas positivas
ds_n = list(path_n.iterdir())     # directorio donde están las críticas negativa

Luego, cada archivo tenemos que convertirlos en cadenas de texto.

In [11]:
def convert_file_to_text(file_path: Path) -> str:
    # Your code here !
    text = []
    f = open(file_path)
    for line in f.readlines():
        text += line
    return ''.join(text)
    #

Por ejemplo, de esta forma se convierte un archivo:

In [12]:
a = convert_file_to_text(ds_p[0])
a

'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers to direct this seem

Luego, hacemos lo mismo para cada archivo, guardándolos en dos listas diferentes, una de críticas positivas y otra de críticas negativas.

In [13]:
texts_p = []    # Lista de críticas positivas
texts_n = []    # Lista de críticas negativas

Para esto se carga el contenido de cada archivo de crítica positiva y se salva en la lista de críticas positivas.

In [14]:
for file in ds_p:
    texts_p.append(convert_file_to_text(file))

Y se hace lo mismo con los archivos de críticas negativas y la lista de críticas negativas.

In [15]:
for file in ds_n:
    texts_n.append(convert_file_to_text(file))

Para comprobar el resultado de este proceso, se comprueba la longitud de las listas, estas deben tener 1000 elementos.

In [16]:
len(texts_p)

1000

In [17]:
len(texts_n)

1000

### Ejercicio 2: Extracción de características

¿Cómo convertimos una cadena de texto en una matriz de características?

Existen varias maneras, una de las más sencillas, es usando `CountVectorizer` de `sklearn`, que convierte una collección de documentos en una matriz con recuentos de tokens.

In [18]:
from sklearn.feature_extraction.text import CountVectorizer 

Para crear la matriz de características con `CountVectorizer`, se usa su método `fit_transform`, que crea un diccionario con el vocabulario del texto, y se retorna una matriz que guarda para cada documento la cantidad de veces que ocurre un término. 

In [19]:
vectorizer = CountVectorizer()
mt = vectorizer.fit_transform(texts_p + texts_n)
mta = mt.toarray()

La matriz de características tiene dos dimensiones, la primera representa la cantidad de instancias (ejemplos) y la segunda la cantidad de características. Busquemos las dimensiones de la matriz:

In [20]:
# matriz de características
mt.shape

(2000, 39659)

Para cada documento, ¿cuál es el por ciento de términos cuya ocurrencia es distinto de cero? Es decir, ¿cuál es el por ciento de los valores en la matriz que son distintos de cero? 

In [27]:
mt.nnz * 100.0 / (mt.shape[0] * mt.shape[1])

0.8407196348874152

El vector de clase dice, para cada instancia cuál es la clase a la que pertenece. La clase positiva (1) representa una crítica positiva y la clase negativa (0) una crítica negativa.

In [29]:
y = [1]*1000 + [0]*1000

### Ejercicio 3: Elección del clasificador

Ahora que ya se tienen la matriz de clasificación y el vector de clases, ¿qué hacemos?, ¿qué clasificador usamos?, ¿por qué?

El Naive Bayes de sklearn se llama Gaussian Naive Bayes porque está diseñado (a diferencia del visto en conferencia) para lidiar con características que sean valores continuos. Gaussian Naive Bayes supone que la probabilidad de las características es gaussiana: $$P(x_i ∣ y)=\dfrac{1}{\sqrt{2\pi \sigma^2_y}}exp(−\dfrac{{(x_i-\mu_y)^2}}{2\sigma^2_y})$$

In [30]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

### Ejercicio 4: Empleo del clasificador elegido

Para comenzar el entrenamiento del clasificador elegido, es necesario dividir la matriz de características y el vector de clase en dos conjuntos, uno de entrenamiento y otro de prueba. En este caso se quiere entrenar en el 60% de los datos.

In [31]:
from sklearn.model_selection import train_test_split

# Your code here !
X_train, X_test, y_train, y_test = train_test_split(mta, y, train_size=0.60)
#

Se comprueba que el 60% del total de instancias (1000 ejemplos) correponden al conjunto de entrenamiento (serían 1200 instancias) y el resto (800) al conjunto de prueba.  

In [32]:
print(X_train.shape)
print(X_test.shape)

(1200, 39659)
(800, 39659)


Primero, se realiza el entrenamiento con todos los datos, y se prueban los resultados obtenidos en este mismo conjunto. 

El entrenamiento en los algoritmos de aprendizaje de máquinas de sklearn se realizan todos mediante la función `fit`, que recibe la matriz de características y el vector de clase. Para obtener la precisión del rendimiento del algoritmo, se utiliza la función `score`, que también recibe la matriz de características y el vector de clase, y retoran el porcentaje total de elementos clasificados correctamente.

In [35]:
naive_bayes = GaussianNB()
naive_bayes.fit(mta,y)
naive_bayes.score(mta,y)

0.9985

Ahora, se hace correctamente, realizando el entrenamiento en el conjunto de entrenamiento y probando los resultados obtenidos en un conjunto de prueba, que nunca antes fue visto por el algoritmo.

In [36]:
# Your code here !
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
naive_bayes.score(X_test, y_test)
#

0.65125

### Ejercicio 5: Empleo del clasificador restante

Ahora probemos los resultados con el otro algoritmo dado en clases.

In [37]:
# Your code here !
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)
#

0.60375

### Ejercicio 6: Experimentos

Para comprobar la eficiencia real del algoritmo es necesario realizar el entrenamiento y prueba del algoritmo utilizado varias veces. Así, se tiene un valor más estadísticamente significativo del rendimiento del algoritmo en el dataset utilizado.

In [38]:
def experiments(Clsf, iterations: int) -> List[float]:
    rs = []
    # Your code here !
    for _ in range(iterations):
        X_train, X_test, y_train, y_test = train_test_split(mta, y, train_size=0.60)
        clf = Clsf()
        clf.fit(X_train, y_train)
        rs.append(clf.score(X_test, y_test))
    #
    return rs

In [39]:
import numpy as np

Se comprueban los resultados obtenidos con el algoritmo de KNN.

In [40]:
# Your code here !
rs1 = experiments(KNeighborsClassifier, 30)

Para tener una idea del rendimiento promedio, se calcula la media de los resultados obtenidos.

In [41]:
# Your code here !
np.mean(rs1)

0.6232916666666667

Y luego se comprueban los resultados con Naive Bayes.

In [42]:
# Your code here !
rs2 = experiments(GaussianNB, 30)

In [43]:
# Your code here !bb
np.mean(rs2)

0.6492916666666666

Por último, comprobamos mediante una prueba de hipótesis si la media de las muestras correspondientes a los resultados de rendimiento son diferentes.

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
# Your code here !
result = ttest_ind(rs1, rs2)
result.pvalue < 0.01

### Tarea

Hacer un Naive Bayes que supere al de sklearn utilizado en clase para el corpus
utilizado (rotten tomatoes). Debe dar un 80%.